In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging

At the first block of every epoch
- Get the list of elected validators (elected_list)
- Get the stakes of validators who were eligible for election in the last epoch (last block)
- Check if any of these validators not present in elected_list

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.t.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":"1"}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    return block, last_block

In [7]:
def getCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [8]:
def getAllValidator():
    method = 'hmy_getAllValidatorAddresses'
    params = []
    return get_information(method, params)['result']
    

In [9]:
def getValidatorByBlock(validator, block):
    method = "hmy_getValidatorInformationByBlockNumber"
    params = [validator, block]
    return get_information(method, params)['result']

In [10]:
def getAllElectedValidator():
    method = "hmy_getElectedValidatorAddresses"
    params =[]
    return get_information(method, params)['result']

In [11]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [0]
    return get_information(method, params)['result']

In [12]:
def E3_test(single):
    logger.info(f"Test-E3: Joining before election start must consider the validator for election")
    try:
        committees = getCommittees()
        slot = committees['external-slot-count']
        iterations = 0
        num = 1
        while iterations < num:
            block, last_block = getCurrentAndLastBlock()
            logger.info(f"current and last block numbers: {block}, {last_block}")
            if block == last_block:
                logger.info(f"current block is the last block in epoch, waiting for the new epoch...")
                new_block = block+1
                while block < new_block:
                    block = getBlockNumber()
                block, last_block = getCurrentAndLastBlock()
                logger.info(f"current and last block numbers: {block}, {last_block}")
            second_last_block = last_block - 1
            while block < second_last_block:
                block = getBlockNumber()
            logger.info(f"second last block in current epoch reached, {block}, wait for 6 seconds to reach the end of the block")
            time.sleep(6)
            logger.info("begin collecting eligible validators...")
            # get top #slots nodes who are eligible to elected next epoch
            validator_infos = getAllValidatorInformation()
            eligible = []
            stake = dict()
            for i in validator_infos:
                if i['epos-status'] == 'currently elected' or i['epos-status'] == 'eligible to be elected next epoch':
                    address = i['validator']['address']
                    eligible.append(address)
                    stake[address] = i['total-delegation']

            if len(eligible) > slot:
                sorted_stake = sorted(stake.items(), key=lambda kv: kv[1], reverse = True)
                eligible = [kv[0] for kv in sorted_stake[:slot]]

            # wait for epoch changes
            new_block = block + 2
            while block < new_block:
                block = getBlockNumber()
            logger.info(f"first block in new epoch reached, {block}, will wait for 5 seconds to begin testing...")
            time.sleep(5)
            # check whether the eligible validators are selected
            validator_infos = getAllValidatorInformation()
            flag = True
            for i in validator_infos:
                if i in eligible:  
                    if i['epos-status'] != 'currently elected':
                        logger.warning(f"Test-E3: Fail")
                        logger.warning(f"Validator {i} joined before election was not considered as the validator for election\n")
                        flag = False
            iterations += 1
    except TypeError as e:
        logger.error(f"error: {e}")
    if single:
        curr_test = None
    else:
        curr_test = E4_test
    if flag:
        logger.info(f"Test E3: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [13]:
E3_test(True)

INFO:economic-test:Test-E3: Joining before election start must consider the validator for election
INFO:economic-test:current and last block numbers: 3489580, 3489791
INFO:economic-test:second last block in current epoch reached, 3489790, wait for 6 seconds to reach the end of the block
INFO:economic-test:begin collecting eligible validators...
INFO:economic-test:first block in new epoch reached, 3489792, will wait for 5 seconds to begin testing...
INFO:economic-test:Test E3: Succeed



(True, None)